# base feature

In [3]:
from multiprocessing import Pool
from multiprocessing import cpu_count
import math
import pandas as pd
import datetime
import numpy as np
import gc

processor=cpu_count()-2
processor

6

### 1. concat 特征的思路

#### 通过对于特征的连结，可以分析交互特征

In [1]:
def concate_feature(data):
    features=[('user_occupation_id','shop_star_level'),('item_collected_level','item_pv_level'),('user_star_level','hour48'),('item_price_level','hour48'),('item_sales_level','context_page_id')]
    con_fea=[]
    def concate_feature(data, f1, f2, name):
        data[name] = data.apply(lambda x: str(x[f1]) + ';' + str(x[f2]), axis=1)
        return data
    for i in features:
        data=concate_feature(data, i[0], i[1], '_con_'.join(i))
        con_fea.append('_con_'.join(i))
    return data,con_fea


### 2. query 特征的抽取

#### 大框架：首先对于每条查询，抽取出原始数据中，instance_id 和它相同的所有观测

In [6]:
data=pd.read_csv('query_xxx.csv')
features=[]
for index,row in data.iterrows():
    feature={}
    feature['instance_id']=row['instance_id']
    if index%100==0:
        print(index)
    col=['user_id','predict_category_property','context_timestamp','day','query1','query','item_id','shop_id','item_brand_id','item_city_id','context_page_id','item_category_list']
    # tmp 就是 instance_id 相同的所有的 数据
    tmp=data[data['user_id']==row['user_id']][['instance_id']+col]
    
    

#### 2.1 query_cnt 信息统计 : 反应一个人对本次查询的关注程度

#### before_query_cnt   : 所有在这条记录之前，predict_category_property 和 这条记录一致的记录的总数
#### before_query_1_cnt : 所有在这条记录之前，query1 和 这条记录一致的记录的总数
#### before_query_all_cnt : 所有在这条记录之前，query 和 这条记录一致的记录的总数
#### after_query_cnt 、before_query_1_cnt 、 before_query_all_cnt 的处理方法类似


In [ ]:
before_query_cnt=len(
    tmp[
            (tmp['predict_category_property']==row['predict_category_property'])& \
            (tmp['context_timestamp']<row['context_timestamp'])& \
            (tmp['day']<=row['day'])
    ]
)
before_query_1_cnt = len(
    tmp[
        (tmp['query1'] == row['query1']) & \
        (tmp['context_timestamp'] < row['context_timestamp']) & \
        (tmp['day'] <= row['day'])
    ]
)
before_query_all_cnt = len(
    tmp[
        (tmp['query'] == row['query']) &  \
        (tmp['context_timestamp'] < row['context_timestamp']) & \
        (tmp['day'] <= row['day'])
    ]
)

after_query_cnt = len(
    tmp[
        (tmp['predict_category_property'] == row['predict_category_property']) & \
        (tmp['context_timestamp'] > row['context_timestamp'])& \
        (tmp['day']<=row['day'])
    ]
)

after_query_1_cnt = len(
    tmp[
        (tmp['query1'] == row['query1']) &\
        (tmp['context_timestamp'] > row['context_timestamp']) & 
        (tmp['day'] <= row['day'])
    ]
)

after_query_all_cnt = len(
    tmp[
        (tmp['query'] == row['query']) & \
        (tmp['context_timestamp'] > row['context_timestamp']) &  \
        (tmp['day'] <= row['day'])
    ]
)

#### 2.2 分属性查询商品统计信息：反应用户有多喜欢查询某个属性
#### before_query_item_cnt     : 所有在这条记录之前，item_id 和 这条记录一致 \ predict_category_property 和这条记录一致 的记录的总数
#### before_query_1_item_cnt   : 所有在这条记录之前，item_id 和 这条记录一致 \ query1 和 这条记录一致 的记录的总数
#### before_query_all_item_cnt : 所有在这条记录之前，item_id 和 这条记录一致 \ query 和 这条记录一致 的记录的总数
#### after_query_item_cnt \ after_query_1_item_cnt \ after_query_all_item_cnt 类似      key = item_id
#### before_query_shop_cnt \ before_query_1_shop_cnt \ before_query_all_shop_cnt \ 类似 key = shop_id
#### after_query_shop_cnt  \ after_query_1_shop_cnt \  after_query_all_shop_cnt \  类似 key = shop_id
#### before_query_brand_cnt \ before_query_all_brand_cnt \ before_query_1_brand_cnt \  类似 key = item_brand_id
#### after_query_brand_cnt  \ after_query_1_brand_cnt \  after_query_all_brand_cnt \  类似 key = item_brand_id
#### before_query_city_cnt \ before_query_all_city_cnt \ before_query_1_city_cnt \  类似 key = item_city_id
#### after_query_city_cnt  \ after_query_1_city_cnt \  after_query_all_city_cnt \  类似 key = item_city_id
#### context_page_id  、 item_category_list 、 也可以做类似的操作

In [8]:
before_query_item_cnt = len(
    tmp[
        (tmp['item_id']==row['item_id'])&\
        (tmp['predict_category_property']==row['predict_category_property'])&\
        (tmp['context_timestamp']<row['context_timestamp'])&(tmp['day']<=row['day'])
    ]
)
before_query_1_item_cnt = len(
    tmp[
        (tmp['item_id'] == row['item_id']) & (tmp['query1'] == row['query1']) &  \
        (tmp['context_timestamp'] < row['context_timestamp']) & (tmp['day'] <= row['day'])
    ]
)
before_query_all_item_cnt = len(
    tmp[
        (tmp['item_id'] == row['item_id']) & (tmp['query'] == row['query']) & \
        (tmp['context_timestamp'] < row['context_timestamp']) & (tmp['day'] <= row['day'])
    ]
)


#### 2.3 查询频率信息
#### before_diff_query_cnt     ：反应在本次查询之前，用户一共查询了多少次
#### before_diff_query_all_cnt ：在本次查询之前，用户一共查询了多少次和 本次查询的query 一致的查询
#### before_diff_query_1_cnt   ：在本次查询之前，用户一共查询了多少次和 本次查询的query1 一致的查询
#### after_diff_query_all_cnt 、 after_diff_query_1_cnt 类似

In [ ]:
before_diff_query_cnt= len(
    set(tmp[
        (tmp['context_timestamp']<row['context_timestamp'])& \
        (tmp['predict_category_property']!=row['predict_category_property'])
        ]
       )
)
before_diff_query_all_cnt = len(
    set(tmp[(tmp['context_timestamp'] < row['context_timestamp']) & \
            (tmp['query'] != row['query'])]))
before_diff_query_1_cnt = len(
    set(tmp[(tmp['context_timestamp'] < row['context_timestamp']) &\
        (tmp['query1'] != row['query1'])]))

#### 2.4 查询时间信息统计
#### query_min_time     ： predict_category_property 和本次查询相同时（相同关键词查询）的最早一次查询时间
#### query_all_min_time ： query 和 本次查询相同时 的最早一次时间
#### query_1_min_time   ： query1 和 本次查询相同时 的最早一次时间

In [ ]:
query_min_time=np.min(
    tmp[
        (tmp['predict_category_property'] == row['predict_category_property'])
    ]['context_timestamp']
)
query_all_min_time = np.min(
    tmp[
        (tmp['query'] == row['query'])
    ]['context_timestamp']
)
query_1_min_time = np.min(tmp[(tmp['query1'] == row['query1'])]['context_timestamp'])

#### 2.5 除了本次关注的信息，用户还关注了什么？
#### before_query_items ： 在 query_min_time（本次查询关键词第一次出现的时间）之前，用户浏览的商品的总数
#### before_query_all_items ：在 query_all_min_time（本次查询内容-query 第一次出现的时间）之前，用户浏览的商品的总数
#### before_query_1_items ： 在 query_1_min_time（本次查询内容-query1 第一次出现的时间）之前，用户浏览的商品的总数
#### 对shop 也可以做类似的操作

In [ ]:
before_query_items= len(set(tmp[(tmp['context_timestamp'] <query_min_time)]['item_id']))
before_query_all_items = len(set(tmp[(tmp['context_timestamp'] < query_all_min_time)]['item_id']))
before_query_1_items = len(set(tmp[(tmp['context_timestamp'] < query_1_min_time)]['item_id']))
before_query_shops = len(set(tmp[(tmp['context_timestamp'] < query_min_time)]['shop_id']))
before_query_all_shops = len(set(tmp[(tmp['context_timestamp'] < query_all_min_time)]['shop_id']))
before_query_1_shops = len(set(tmp[(tmp['context_timestamp'] < query_1_min_time)]['shop_id']))

#### 在本次查询之后，用户还关注了什么
#### query_max_time 、 query_all_max_time 、 query_1_max_time 、 
#### after_query_items 、 after_query_all_items 、 after_query_1_items 、shops 都可以来一发

In [ ]:
query_max_time = np.max(tmp[(tmp['predict_category_property'] == row['predict_category_property'])]['context_timestamp'])
query_all_max_time = np.max(tmp[(tmp['query'] == row['query'])]['context_timestamp'])
query_1_max_time = np.max(tmp[(tmp['query1'] == row['query1'])]['context_timestamp'])
after_query_items = len(set(tmp[(tmp['context_timestamp'] > query_max_time)]['item_id']))
after_query_all_items = len(set(tmp[(tmp['context_timestamp'] > query_all_max_time)]['item_id']))
after_query_1_items = len(set(tmp[(tmp['context_timestamp'] > query_1_max_time)]['item_id']))
after_query_shops = len(set(tmp[(tmp['context_timestamp'] > query_max_time)]['shop_id']))
after_query_all_shops = len(set(tmp[(tmp['context_timestamp'] > query_all_max_time)]['shop_id']))
after_query_1_shops = len(set(tmp[(tmp['context_timestamp'] > query_1_max_time)]['shop_id']))

##